In [23]:
import warnings
warnings.simplefilter("ignore")

In [24]:
import datasets

from peft import PrefixTuningConfig, TaskType, get_peft_model

from opacus.validators import ModuleValidator
from opacus import PrivacyEngine

import torch
import torch.nn as nn
import numpy as np

from tqdm.notebook import tqdm
from torch.optim import SGD
from torch.utils.data import DataLoader

from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, get_linear_schedule_with_warmup

from sklearn.metrics import accuracy_score

In [25]:
model_name = "prajjwal1/bert-tiny"
EPOCHS = 3
BATCH_SIZE = 32
LR = 5e-5

In [26]:
dataset = datasets.load_dataset('glue', 'mnli')
num_labels = dataset["train"].features["label"].num_classes

In [27]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [28]:
def preprocess_function(examples):
    result = {}
    result["input_ids"] = examples["input_ids"]
    result["attention_mask"] = examples["attention_mask"]
    result["labels"] = examples["label"]
    return result

In [29]:
processed_datasets = tokenized_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [30]:
processed_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
processed_datasets = processed_datasets.remove_columns(["premise", "hypothesis", "idx", "label"])

In [31]:
train_dataloader = DataLoader(processed_datasets["train"], batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(processed_datasets["validation_matched"], batch_size=BATCH_SIZE, shuffle=False)

In [32]:
EPSILON = np.inf
DELTA = 1
MAX_GRAD_NORM = 0
MAX_PHYSICAL_BATCH_SIZE = int(BATCH_SIZE/4)

In [33]:
config = AutoConfig.from_pretrained(model_name)
config.num_labels = num_labels

peft_config = PrefixTuningConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    num_virtual_tokens=10,
)

model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)
model = get_peft_model(model, peft_config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 128, padding_idx=0)
        (position_embeddings): Embedding(512, 128)
        (token_type_embeddings): Embedding(2, 128)
        (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-1): 2 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=128, out_features=128, bias=True)
                (key): Linear(in_features=128, out_features=128, bias=True)
                (value): Linear(in_features=128, out_features=128, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features

In [34]:
errors = ModuleValidator.validate(model, strict=False)
print(errors)

[]


In [35]:
optimizer = SGD(params=model.parameters(), lr=LR)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_dataloader)*EPOCHS
)

In [36]:
privacy_engine = PrivacyEngine(accountant="rdp")

model, optimizer, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
    batch_first=True,
)

In [37]:
print(f"Using Sigma = {optimizer.noise_multiplier:.3f} | C = {optimizer.max_grad_norm} | Initial DP (ε, δ) = ({privacy_engine.get_epsilon(DELTA)}, {DELTA})")

Using Sigma = 10.000 | C = 0 | Initial DP (ε, δ) = (0, 1)


In [38]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Trainable Parameters: {trainable_params} || All Parameters: {all_param} || Trainable Parameters (%): {100 * trainable_params / all_param:.2f}"
    )

print_trainable_parameters(model)

Trainable Parameters: 5507 || All Parameters: 4391814 || Trainable Parameters (%): 0.13


In [39]:
def train(model, train_dataloader, optimizer, epoch, device):
    model.train()
    criterion = nn.CrossEntropyLoss()

    losses = []
    epsilon = []

    for i, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Training Epoch: {epoch}"):
        
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()

        outputs = model(**batch)
        loss = criterion(outputs.logits, batch["labels"])
        loss.backward()

        optimizer.step()
        losses.append(loss.item())

        if i % 8000 == 0:
            epsilon = privacy_engine.get_epsilon(DELTA)

            print(f"Training Epoch: {epoch} | Loss: {np.mean(losses):.6f} | ε = {epsilon:.2f}")                   

In [40]:
def test(model, test_dataloader, device):
    model.eval()
    eval_loss = 0
    eval_preds = []

    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc="Test"):
            batch = {k: v.to(device) for k, v in batch.items()}

            outputs = model(**batch)
        loss = outputs.loss

        eval_loss += loss.detach().float()
        preds = outputs.logits.argmax(dim=-1)

        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )
        acc = accuracy_score(preds.cpu().numpy(), batch["labels"].cpu().numpy())

    print(
        f"Test set: Loss: {eval_loss:.4f}, Accuracy: {acc*100:.2f}%"
    )

    return eval_loss, acc

In [41]:
for epoch in tqdm(range(EPOCHS), desc=f'Training {EPOCHS} Epochs'):
    train(model, train_dataloader, optimizer, epoch + 1, device)

Training 3 Epochs:   0%|          | 0/3 [00:00<?, ?it/s]

Training Epoch: 1:   0%|          | 0/12272 [00:00<?, ?it/s]

Training Epoch: 1 | Loss: 1.085112 | ε = -3.35
Training Epoch: 1 | Loss: 1.102777 | ε = -3.35


Training Epoch: 2:   0%|          | 0/12272 [00:00<?, ?it/s]

Training Epoch: 2 | Loss: 1.089408 | ε = -3.35
Training Epoch: 2 | Loss: 1.103013 | ε = -3.35


Training Epoch: 3:   0%|          | 0/12272 [00:00<?, ?it/s]

Training Epoch: 3 | Loss: 1.107631 | ε = -3.35
Training Epoch: 3 | Loss: 1.103340 | ε = -3.35


In [42]:
final_epsilon = privacy_engine.get_epsilon(DELTA)
print(f"Final DP Guarantee (ε, δ)-DP = ({final_epsilon:.2f}, {DELTA})")

Final DP Guarantee (ε, δ)-DP = (-3.35, 1)


In [43]:
test(model, test_dataloader, device)

Test:   0%|          | 0/307 [00:00<?, ?it/s]

Test set: Loss: 1.0757, Accuracy: 52.17%


(tensor(1.0757, device='cuda:0'), 0.5217391304347826)